In [6]:
## Load necessary libraries
%matplotlib inline
from constructIDF import *
import pandas as pd
import numpy as np
import itertools
import argparse
import matplotlib.pyplot as plt

from matplotlib import rcParams

rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

### Step 1: Read file with daily rainfall records from MACA.

The data used in this example was downloaded from the MACA portal. The data corresponds to downscaled daily rainfall observations from 10 different GCM extracted from the closest grid cell center to Ann Arbor lat, lon coordinates (Latitude:42.2981, Longitude:-83.6639)

In [1]:
"[*]"

# Specify path to hourly rainfall time series

## REMEMBER to specify the path to where the historical data from MACA was stored.

historical_path = ""

In [3]:
## Reading data in
## Historical_data is the historical MACA data for Ann Arbor.

historical_data = pd.read_csv(historical_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

### Step 2: Construct Annual Maximum Series

We only have daily observations, so we can only do starting
from 24. We will only create 24-hour IDF curves in this tutorial.

In [4]:
## Reformatting data so that it is in the format 
## required by the methods we used in HW1

historical_data['year'] = historical_data["yyyy-mm-dd"].dt.year

#The following line of code gets the maximum daily rainfall 
# on each year
historicalAMS = historical_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()


cols = list(historicalAMS)
cols.insert(0, cols.pop(cols.index('year')))
historicalAMS = historicalAMS.loc[:, cols]
out = historicalAMS.reset_index(drop=True)

In [5]:
#Take a look

#########################
#  Historical Model     #
#     AMS TABLE         #
#                       #
#########################

out

,year,pr_bcc-csm1-1_historical(mm),pr_bcc-csm1-1-m_historical(mm),pr_BNU-ESM_historical(mm),pr_CanESM2_historical(mm),pr_CCSM4_historical(mm),pr_CNRM-CM5_historical(mm),pr_CSIRO-Mk3-6-0_historical(mm),pr_GFDL-ESM2M_historical(mm),pr_GFDL-ESM2G_historical(mm),pr_HadGEM2-CC365_historical(mm)
0,1950,32.170700,28.330273,25.961569,41.700222,23.778496,40.512627,30.651091,34.664284,40.512627,40.512627
1,1951,38.417217,30.189684,28.345760,51.715496,38.092495,29.494617,34.913235,46.009525,38.343445,47.898281
2,1952,40.362488,36.423500,53.989502,47.898281,40.362488,39.591415,24.854980,46.715599,42.725613,33.969921
3,1953,35.092014,34.664284,52.026752,41.975277,38.320995,30.128036,27.875061,35.953228,27.804882,36.172832
4,1954,44.760929,51.715496,42.725613,32.958679,40.126492,45.494953,39.536640,38.991837,58.161530,38.063091
5,1955,46.608860,28.903296,47.898281,41.021931,35.660465,35.062305,40.512627,52.803215,40.023609,65.595009
6,1956,46.715599,65.595009,36.172832,28.345760,35.062305,38.320995,34.205025,32.816669,41.021931,27.459637
7,1957,32.048145,41.975277,44.760929,47.190266,36.778805,35.092014,26.854403,38.343445,45.494953,37.490189
8,1958,65.595009,52.026752,31.787653,34.985138,26.592812,36.684879,42.941067,32.277603,28.192711,41.700222
9,1959,38.063091,46.715599,39.570255,31.305250,30.741625,52.026752,33.897297,42.725613,30.758911,25.397488


### Step 3: Fit Generalized Extreme Value and obtain rainfall depths

The next step is to fit a generalized extreme value distribution to each duration's AMS. Once the parameters (location, scale and shape) are estimated, these are used to retrieve the return levels (in this case, rainfall depth) for different quantiles feed into the inverse of the CDF. Usually, the quantiles are equal to the inverse of the average recurrence interval (ARI) (e.g. 1/2 = 2-year).

`constructIDF` has one method that merges all these steps, but we need to specify if we want to construct confidence intervals. The method implemented in `constructIDF` is bootstrapping, so we also need to specify the number of bootsrapped samples. Default value is 1000, and using a smaller number is not recommended.

Other specification is the confidence level, alpha, used to estimate the confidence intervals. Default is 0.9 (90% confidence interval).


In this tutorial, we will compute confidence intervals at a 90% confidence level using 1000 bootstrapped samples.

```python
ci = True
alpha = 0.9
number_bootstrap = 1000
```

In [7]:
# Specifying values #

ci = True
alpha = 0.9
number_bootstrap = 100

In [8]:
# Feeding the data and our specifications to the method.

data = IDF(out, ci, number_bootstrap, alpha)

In [9]:
# Construct IDF from the data we feed above and our specifications.
# Some errors will be displayed, no worries. This will take long time because
# of the number of bootsrapped samples.
# The constructed IDF is by default for the following ARI:
# 2-, 5-, 10-, 25-, 50-, 100-, 200-year

data.construct_IDF()


/Users/tanialopez/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:2552: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)


In [10]:
######################################
#                                    # 
#          Historical Model          #
#             IDF TABLE              #
#                                    #
######################################

## We can access the dataframe with confidence bounds:

## Note that we no longer have "DURATIONS" by column. 
## We only did for one duration since we only have daily data.
## Now, each column corresponds to the 24-hour historical IDF
## curve for each model.

data.idf

,pr_bcc-csm1-1_historical(mm),pr_bcc-csm1-1-m_historical(mm),pr_BNU-ESM_historical(mm),pr_CanESM2_historical(mm),pr_CCSM4_historical(mm),pr_CNRM-CM5_historical(mm),pr_CSIRO-Mk3-6-0_historical(mm),pr_GFDL-ESM2M_historical(mm),pr_GFDL-ESM2G_historical(mm),pr_HadGEM2-CC365_historical(mm)
L2-yr,37.179632,36.856761,37.263662,36.211978,36.236925,36.308653,34.510024,36.901454,36.035395,37.061268
L5-yr,44.670891,44.526018,44.855749,43.827948,44.272262,43.556233,43.359643,44.621320,43.395854,44.191385
L10-yr,49.329410,49.574443,49.013967,49.260871,48.987648,48.579716,49.460903,49.419522,48.066798,48.731540
L25-yr,54.331800,54.791440,54.747576,54.729435,54.301353,53.486110,55.957525,55.753955,54.412192,54.015279
L50-yr,56.724633,57.810062,58.409701,58.365703,57.762701,56.688999,59.758957,59.959101,58.395860,57.612257
L100-yr,58.927360,60.088218,61.413128,61.130355,60.748303,59.647612,63.388108,64.005502,61.848509,60.725136
L200-yr,60.865839,62.453626,63.955123,64.233152,63.386529,62.374525,66.578361,67.590498,65.110204,63.153346
2-yr,39.239200,38.772434,39.359640,38.609688,38.382153,38.288067,37.123625,39.147372,37.998193,38.692638
5-yr,47.038686,47.229581,47.380359,47.275351,46.986564,46.929864,46.391243,47.413836,46.627830,47.180445
10-yr,52.285479,52.469840,52.736323,52.831299,52.367155,52.254449,53.178611,53.128375,52.434946,52.375800


### Step 3: Obtain IDF values from future data

Now, we need to repeat the same process but inputing the
future model data.

In [10]:
"[*]"

"""
Specify here the path to the data from RCP 8.5 or RCP 4.5 downloaded from MACA
"""
future_path = " "

In [12]:
## Reading data in
## Historical_data is the historical MACA data for Ann Arbor.

future_data = pd.read_csv(future_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

In [13]:
## Reformatting data so that it is in the format 
## required by the methods we used in HW1

future_data['year'] = future_data["yyyy-mm-dd"].dt.year

#The following line of code gets the maximum daily rainfall 
# on each year
futureAMS = future_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()


cols = list(futureAMS)
cols.insert(0, cols.pop(cols.index('year')))
futureAMS = futureAMS.loc[:, cols]
future_out = futureAMS.reset_index(drop=True)

In [14]:
#Take a look

#########################
#      Future Model     #
#       AMS TABLE       #
#                       #
#########################

future_out

,year,pr_bcc-csm1-1_rcp85(mm),pr_bcc-csm1-1-m_rcp85(mm),pr_BNU-ESM_rcp85(mm),pr_CanESM2_rcp85(mm),pr_CCSM4_rcp85(mm),pr_CNRM-CM5_rcp85(mm),pr_CSIRO-Mk3-6-0_rcp85(mm),pr_GFDL-ESM2M_rcp85(mm),pr_GFDL-ESM2G_rcp85(mm),pr_HadGEM2-CC365_rcp85(mm)
0,2006,41.762489,65.705200,41.413303,44.896496,40.287437,49.113220,32.486240,49.317863,39.887020,66.769920
1,2007,38.984734,28.114281,31.947811,46.017780,29.723171,33.605335,54.532627,43.626713,77.920204,45.774925
2,2008,45.069363,34.916054,36.909401,42.551731,62.972191,42.009243,62.468037,31.248665,52.733070,35.564102
3,2009,48.521442,42.810760,33.440716,79.665039,27.621651,42.226654,39.272945,41.084148,32.946880,33.020657
4,2010,56.192223,34.005508,30.876858,47.956028,40.311985,38.033913,57.396362,29.048100,35.026276,25.479580
5,2011,32.948307,78.111465,30.275681,53.245522,32.609035,50.956348,33.942802,38.154999,42.439518,41.509102
6,2012,44.667282,55.735611,48.518406,33.023602,47.354401,42.440079,106.742500,35.227810,60.910507,25.867962
7,2013,36.793396,34.378925,42.378456,22.895462,31.897778,39.370762,50.096195,53.535370,66.857185,47.343544
8,2014,37.680214,52.181297,37.767151,43.155308,30.192486,37.534046,36.844730,47.618202,39.964985,50.403439
9,2015,36.773064,51.475437,80.702110,35.786327,42.850693,43.854259,95.317520,38.230331,40.249603,41.111614


In [15]:
# Feeding the data and our specifications to the method,
# same as when we calculated the historical model IDF values.

future_data = IDF(future_out, ci, number_bootstrap, alpha)

In [16]:
# Construct IDF from the data we feed above and our specifications.
# Some errors will be displayed, no worries. This will take long time because
# of the number of bootsrapped samples.
# The constructed IDF is by default for the following ARI:
# 2-, 5-, 10-, 25-, 50-, 100-, 200-year

future_data.construct_IDF()


In [17]:
######################################
#                                    # 
#          Future Model              #
#           IDF TABLE                #
#                                    #
######################################

## We can access the dataframe with confidence bounds:

## Note that we no longer have "DURATIONS" by column. 
## We only did for one duration since we only have daily data.
## Now, each column corresponds to the 24-hour historical IDF
## curve for each model.

future_data.idf

,pr_bcc-csm1-1_rcp85(mm),pr_bcc-csm1-1-m_rcp85(mm),pr_BNU-ESM_rcp85(mm),pr_CanESM2_rcp85(mm),pr_CCSM4_rcp85(mm),pr_CNRM-CM5_rcp85(mm),pr_CSIRO-Mk3-6-0_rcp85(mm),pr_GFDL-ESM2M_rcp85(mm),pr_GFDL-ESM2G_rcp85(mm),pr_HadGEM2-CC365_rcp85(mm)
L2-yr,44.290819,42.464687,39.458595,41.172890,39.701816,41.406511,46.165407,43.254898,44.688878,44.512947
L5-yr,55.537796,54.710998,48.553148,56.241234,49.714165,51.080003,60.740722,53.922763,56.950304,56.787018
L10-yr,63.572100,63.754381,55.187264,69.529480,56.769117,57.102758,70.771227,61.194256,64.914282,64.963155
L25-yr,73.276871,74.487515,62.721260,91.072336,64.463204,64.662137,83.649040,69.596028,74.785843,74.949838
L50-yr,80.815371,82.234487,67.806306,110.235781,69.887099,70.044111,91.854216,76.234845,81.155169,81.547712
L100-yr,87.693600,89.253857,72.935817,132.975479,75.850184,75.616722,100.178850,82.786786,86.974366,87.587508
L200-yr,94.859071,95.736761,78.183744,160.187778,80.119253,80.398496,106.608781,88.282555,92.480276,92.382896
2-yr,46.296216,44.444235,41.356122,43.729626,41.652636,43.293611,48.797437,45.264550,47.027721,46.302502
5-yr,59.603843,58.104875,51.045306,61.222422,52.616993,54.247676,64.042444,57.316113,61.141276,60.327608
10-yr,69.489478,67.664940,57.929077,77.371291,60.150218,61.696072,74.791242,66.225338,71.959605,70.584086


In [18]:
## Find change factors by finding the ratio between
## historical IDF model values and future IDF model values.

## Find change factor

change_factors = pd.DataFrame(future_data.idf.values/data.idf.values)

In [19]:
## Make table look pretty and understandable 
change_factors.columns = [x.rstrip("_rcp85(mm)") for x in future_data.idf.columns]
change_factors['return_period'] = future_data.idf.index
change_factors.set_index('return_period', inplace=True)

In [20]:
####################################
#                                  # 
#          CHANGE FACTORS          #
#             TABLE                #
#                                  #
####################################

change_factors

,pr_bcc-csm1-1,pr_bcc-csm1-1-,pr_BNU-ESM,pr_CanESM2,pr_CCSM4,pr_CNRM-CM,pr_CSIRO-Mk3-6-0,pr_GFDL-ESM2M,pr_GFDL-ESM2G,pr_HadGEM2-CC36
return_period,,,,,,,,,,
L2-yr,1.191266,1.152155,1.058903,1.136996,1.095618,1.140403,1.337739,1.172173,1.240138,1.201064
L5-yr,1.243266,1.228742,1.082429,1.283228,1.122919,1.172737,1.400858,1.208453,1.312344,1.285025
L10-yr,1.288726,1.286033,1.125950,1.411455,1.158846,1.175444,1.430852,1.238261,1.350501,1.333082
L25-yr,1.348692,1.359474,1.145645,1.664047,1.187138,1.208952,1.494867,1.248271,1.374432,1.387567
L50-yr,1.424696,1.422494,1.160874,1.888708,1.209900,1.235586,1.537079,1.271447,1.389742,1.415458
L100-yr,1.488164,1.485380,1.187626,2.175277,1.248598,1.267724,1.580404,1.293432,1.406248,1.442360
L200-yr,1.558494,1.532926,1.222478,2.493849,1.263979,1.288964,1.601253,1.306139,1.420365,1.462835
2-yr,1.179846,1.146284,1.050724,1.132608,1.085208,1.130734,1.314458,1.156260,1.237630,1.196675
5-yr,1.267124,1.230264,1.077352,1.295018,1.119831,1.155931,1.380486,1.208848,1.311261,1.278657


We can construct the future IDF at Ann Arbor by updating the historical curve (from Homework 1 Part I) with the climate signal that we estimated from the downscaled projections. 
Note that there are two options here:

1. Assume equal change in all storm durations to the estimated change in the 24-h rainfall depth (the change factors we computed above). 
2. Update the 24-hour only.

Option 1 entails a strong assumption, and there is evidence that shorter duration extremes will change substantially more than longer duration extremes You can read this paper here: 

    Prein, A. F., Rasmussen, R. M., Ikeda, K., Liu, C., Clark, M. P., & Holland, G. J. (2017). The future intensification of hourly precipitation extremes. Nature Climate Change, 7(1), 48–52. https://doi.org/10.1038/nclimate3168
    

Therefore, in this homework we will go by Option 2.


In [4]:
"[*]"

# Loading idf values from historical 24-hour duration IDF curve that you generated 
#using the tutorial Historical_IDF_Curves:

# Note that there is no "/" at the end of path_canvas

path_24h_Historical = "" # specify here

name_file = "24H_Pittsburgh.csv"

In [7]:
observed_idf = pd.read_csv("{}/{}".format(path_24h_Historical, name_file), index_col=0)

In [23]:
observed_idf

,24H
L2-yr,1.981448
L5-yr,2.540663
L10-yr,2.891310
L25-yr,3.272468
L50-yr,3.534629
L100-yr,3.756192
L200-yr,3.958497
2-yr,2.116097
5-yr,2.747861
10-yr,3.188775


In [24]:
######################################
#                                    # 
#            Future                  #
#           IDF TABLE                #
#                                    #
######################################

"""
This line of code is multiplying the historical IDF curve values
with the CHANGE FACTORS TABLE computed in the cells above
"""


updated_idf = change_factors.multiply(observed_idf["24H"], axis="index")

In [25]:
## Take a look

updated_idf

,pr_bcc-csm1-1,pr_bcc-csm1-1-,pr_BNU-ESM,pr_CanESM2,pr_CCSM4,pr_CNRM-CM,pr_CSIRO-Mk3-6-0,pr_GFDL-ESM2M,pr_GFDL-ESM2G,pr_HadGEM2-CC36
return_period,,,,,,,,,,
L2-yr,2.360431,2.282935,2.098161,2.252900,2.170910,2.259650,2.650661,2.322601,2.457270,2.379846
L5-yr,3.158720,3.121820,2.750086,3.260249,2.852959,2.979529,3.559109,3.070272,3.334225,3.264814
L10-yr,3.726106,3.718320,3.255469,4.080952,3.350581,3.398574,4.137036,3.580195,3.904718,3.854354
L25-yr,4.413552,4.448834,3.749085,5.445540,3.884872,3.956257,4.891904,4.084926,4.497784,4.540770
L50-yr,5.035772,5.027990,4.103259,6.675883,4.276548,4.367337,5.433003,4.494095,4.912222,5.003118
L100-yr,5.589831,5.579374,4.460950,8.170759,4.689972,4.761815,5.936302,4.858380,5.282139,5.417781
L200-yr,6.169296,6.068082,4.839177,9.871894,5.003458,5.102359,6.338554,5.170346,5.622512,5.790626
2-yr,2.496669,2.425649,2.223434,2.396708,2.296407,2.392743,2.781520,2.446759,2.618946,2.532280
5-yr,3.481880,3.380595,2.960412,3.558528,3.077138,3.176337,3.793382,3.321745,3.603164,3.513571


### Step 4: Generate FUTURE IDF curves


#### For this homework, you will have to perform some statistics, you can directly do them here if you are a python-user. Otherwise, you need to save the data above as a .csv and open in Excel. To  save the data above uncomment the following cell and specify the folder where to save in the save_path variable.

Example: save_path = "\Users\tanialopez\Downloads\"

We can call the `plot_IDF` method to create the IDF curves and plot them.
We need to pass the path where the original data was stored, a path where to store
the figure and its format.

In [9]:
"[*]"

save_path = ""

In [ ]:
# This part generates a plot and saves in your folder specified above

# Hard coded params
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
idf_transposed = updated_idf.transpose()
dfmean = idf_transposed.drop([x for x in idf_transposed.columns if (
                x[:1] == 'L' or x[:1] == 'U')], axis=1)

dfmean = dfmean.transpose()
fig, axs = plt.subplots(figsize=(13, 10))
a1 = dfmean.plot(ax=axs)
fill_alpha = 0.3

legend = plt.legend(title='GCM Model', fontsize=13)
plt.setp(legend.get_title(), fontsize=15)
plt.ylabel('Precipitation Depth (in)', {'fontsize': 18})
plt.xlabel('Return Period', {'fontsize': 18})
plt.title('Future (2006-2099) 24-hour IDF curves', fontsize=22)
plt.grid()

plt.savefig("{}/Figure.{}".format(save_path,
                                          'png'), bbox_inches='tight')